In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install MTCNN


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.3 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from sklearn import svm
import numpy as np
import os

from sklearn.svm import SVC

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from mtcnn import MTCNN

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt

In [4]:
train = ImageDataGenerator(rescale=1)

data = train.flow_from_directory(r'/content/drive/MyDrive/PRML_images/Dataset', target_size=(192,128),
                                       batch_size=2431, shuffle=True,
                                       class_mode="sparse")

Found 2431 images belonging to 10 classes.


In [ ]:
# data_temp = data[0]
# labels = data_temp[1][:40]
# images = data_temp[0][:40]

data_temp = data[0]
labels = data_temp[1][:]
images = data_temp[0][:]

In [ ]:
print(np.array(labels).shape)

(2431,)


In [ ]:
images = np.array(images)
labels = np.array(labels)

In [ ]:
print(np.array(images).shape)

(2431, 192, 128, 3)


In [ ]:
!pip install mtcnn

In [ ]:
mtcnn_detector = MTCNN()
faces_ccord = []
for img in images:
  faces_ccord.append(mtcnn_detector.detect_faces(img))

In [ ]:
img_np = images.copy()

faces = []
exclude_label = []
print(faces_ccord[0])

for i in range(len(faces_ccord)):
    if len(faces_ccord[i]) == 0:
      exclude_label.append(i)
      continue
    x, y, width, height = faces_ccord[i][0]['box']
    x, y = abs(x), abs(y)
    x2, y2 = x + width, y + height
    faces.append(img_np[i][y:y2, x:x2, :])

In [ ]:
print(len(images), len(faces), len(exclude_label))

In [ ]:
plt.imshow(faces[0]/255.)

In [ ]:
# Preprocessing
faces_new = np.array(faces)
faces_new = faces_new/255.
exclude_label = np.array(exclude_label)
label_new = [labels[i] for i in range(len(labels)) if i not in exclude_label]

In [ ]:
print(len(label_new), len(faces), len(exclude_label), len(labels))

In [ ]:
import cv2
print(np.array(faces[3]).shape)
for i in range(len(faces)):
  faces[i] = cv2.resize(faces[i], (32, 48))


print(faces[3].shape)

In [ ]:
# Load the pre-trained VGG16 model without the top classification layers
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 32, 3))

In [ ]:
from keras.applications import VGG16
from keras.layers import Flatten
from keras.models import Model

base_layers = vgg16_model.layers[:10]
flatten_layer = Flatten()

# Combine the first 10 layers and the Flatten layer to create a new model
new_model = Model(inputs=vgg16_model.input, outputs=flatten_layer(base_layers[-1].output))

# Create a base model containing only the selected layers
base_model = Model(inputs=vgg16_model.input, outputs=base_layers[-1].output)

print(new_model.summary())

In [ ]:
features = []
for face in faces:
  features.append(new_model(face.reshape((-1, face.shape[0], face.shape[1], face.shape[2]))))

In [ ]:
new_model.save('best_model.h5')
base_model.save('base_mode.h5')

In [ ]:
features = np.squeeze(features, axis=1)
# Do it on first place before extracting features
X_train, X_test, y_train, y_test = train_test_split(features, label_new, test_size=0.16, random_state=42)

In [ ]:
!pip install joblib

import joblib

In [ ]:

# Convert data and labels to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

# Train an SVM classifier
svm_classifier = SVC(kernel='linear', C=1.0)
svm_classifier.fit(X_train, y_train)

joblib.dump(svm_classifier, 'svm_classifier_model.pkl')

In [ ]:
## Prediction Step

In [ ]:
predicted_label = svm_classifier.predict(X_test)

for i in range(len(predicted_label)):
  predicted_person = predicted_label[i]
  print("Predicted Person:", predicted_person)
  print("Actualo Person:", y_test[i])
  print("--"*50)

accuracy = accuracy_score(y_test, predicted_label)

print(f'Accuracy: {accuracy:.2f}')

Predicted Person: 7.0
Actualo Person: 7.0
----------------------------------------------------------------------------------------------------
Predicted Person: 9.0
Actualo Person: 9.0
----------------------------------------------------------------------------------------------------
Predicted Person: 8.0
Actualo Person: 8.0
----------------------------------------------------------------------------------------------------
Predicted Person: 2.0
Actualo Person: 2.0
----------------------------------------------------------------------------------------------------
Predicted Person: 5.0
Actualo Person: 5.0
----------------------------------------------------------------------------------------------------
Predicted Person: 0.0
Actualo Person: 0.0
----------------------------------------------------------------------------------------------------
Predicted Person: 4.0
Actualo Person: 4.0
----------------------------------------------------------------------------------------------------

In [ ]:
img = imread('path to image');

mtcnn_detector = MTCNN()
faces_ccord = []
faces_ccord.append(mtcnn_detector.detect_faces(img))

In [ ]:
img_np = images.copy()

faces = []
exclude_label = []
print(faces_ccord[0])

for i in range(len(faces_ccord)):
    if len(faces_ccord[i]) == 0:
      exclude_label.append(i)
      continue
    x, y, width, height = faces_ccord[i][0]['box']
    x, y = abs(x), abs(y)
    x2, y2 = x + width, y + height
    faces.append(img_np[i][y:y2, x:x2, :])

In [ ]:
features = []
for face in faces:
  features.append(new_model(face.reshape((-1, face.shape[0], face.shape[1], face.shape[2]))))

In [ ]:
predicted_label = svm_classifier.predict(features)